# Dense Neural Network klasifikator

In [46]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix
from torchvision import datasets
from torchinfo import summary
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

import cv2 as cv

import matplotlib.pyplot as plt

from os import listdir, walk                  
from os.path import isfile, join

import io

from tqdm.auto import tqdm

from pathlib import Path

from timeit import default_timer as timer
from collections import OrderedDict
from typing import Tuple, Dict, List


from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay


In [47]:
SEED = 12
IMG_HEIGHT = 48
IMG_WIDTH = 48
BATCH_SIZE = 64
EPOCHS = 20
FINE_TUNING_EPOCHS = 10
LR = 0.01
NUM_CLASSES = 7
EARLY_STOPPING_CRITERIA=3

## 1. Učitavanje dataseta

In [48]:
def walk_through_dir(dir_path: Path) -> dict:
    """Prints dir_path content"""
    counts = dict()
    classes = dict()
    i=-1
    for dirpath, dirnames, filenames in walk(dir_path):
        i = i+1
        print(f"There are {len(dirnames)} directiories and {len(filenames)} images in '{dirpath}' folder ")
        if (i>0):
            c = dirpath.split('\\')
            counts[c[-1]] = len(filenames)
            classes[c[-1]] = i
    return (counts, classes)

In [49]:
data_path = Path("Kaggle/CV/Emotions_detection")
images_path = data_path / "emotions_dataset"
train_path = images_path / "train"
test_path = images_path / "test"

ctrain, classes = walk_through_dir(train_path)
ctest, _ = walk_through_dir(test_path)

def make_datasets(counts, dataset_type):
    x = []
    y = []

    for label, count in counts.items():
        for i in range(0, count):
            img = cv.imread(f"Kaggle/CV/Emotions_detection/emotions_dataset/{dataset_type}/{label}/im{i}.png")
            img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
            if type(img) is None:
                print("ALOOOOOOOO")
            x.append(img)
            y.append(label)

    x = np.array(x).astype(np.float32)
    return (torch.from_numpy(x), y)

x_train, y_tr = make_datasets(ctrain, "train")
x_test, y_te = make_datasets(ctest, "test")

print(len(x_train))
print(len(x_test))

print(classes)

def replace_label(y):
    y_new = [classes[label] for label in y]
    return torch.from_numpy(np.array(y_new))

y_train = replace_label(y_tr)
y_test  = replace_label(y_te)

print(y_train)

There are 7 directiories and 0 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train' folder 
There are 0 directiories and 3995 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\angry' folder 
There are 0 directiories and 436 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\disgusted' folder 
There are 0 directiories and 4097 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\fearful' folder 
There are 0 directiories and 7215 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\happy' folder 
There are 0 directiories and 4965 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\neutral' folder 
There are 0 directiories and 4830 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\sad' folder 
There are 0 directiories and 3171 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\surprised' folder 
There are 7 directiories and 0 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\test' folder 
Th

In [50]:
def make_dataloaders(images_path, batch_size):
    train_transform = transforms.Compose(
        [transforms.Resize(size=(IMG_HEIGHT, IMG_WIDTH)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor()]
    )

    val_transform = transforms.Compose(
        [transforms.Resize(size=(IMG_HEIGHT, IMG_WIDTH)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor()]
    )

    train_dir = images_path / "train"
    val_dir = images_path / "test"

    train_data = datasets.ImageFolder(
        root=train_dir,
        transform=train_transform
    )

    val_data = datasets.ImageFolder(
        root=val_dir,
        transform=val_transform
    )

    BATCH_SIZE = batch_size

    train_dataloader = DataLoader(
        dataset=train_data,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=4
    )

    val_dataloader = DataLoader(
        dataset=val_data,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=4
    )
    return (train_dataloader, val_dataloader)
BATCH_SIZE = 64
#image_batch, label_batch = next(iter(train_dataloader))
#image_batch.shape, label_batch.shape

## 2. Mreža

In [51]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


### 2.1 Model

In [52]:
model = torchvision.models.densenet169(weights=torchvision.models.DenseNet169_Weights.IMAGENET1K_V1).to(device)
for param in model.parameters():
    param.requires_grad = False

torchvision.models.resnet152(weights = torchvision.models.ResNet152_Weights.DEFAULT).to(device)

summary(
    model=model,
    input_size=(BATCH_SIZE, 3, IMG_HEIGHT, IMG_WIDTH),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
DenseNet (DenseNet)                           [64, 3, 48, 48]      [64, 1000]           --                   False
├─Sequential (features)                       [64, 3, 48, 48]      [64, 1664, 1, 1]     --                   False
│    └─Conv2d (conv0)                         [64, 3, 48, 48]      [64, 64, 24, 24]     (9,408)              False
│    └─BatchNorm2d (norm0)                    [64, 64, 24, 24]     [64, 64, 24, 24]     (128)                False
│    └─ReLU (relu0)                           [64, 64, 24, 24]     [64, 64, 24, 24]     --                   --
│    └─MaxPool2d (pool0)                      [64, 64, 24, 24]     [64, 64, 12, 12]     --                   --
│    └─_DenseBlock (denseblock1)              [64, 64, 12, 12]     [64, 256, 12, 12]    --                   False
│    │    └─_DenseLayer (denselayer1)         [64, 64, 12, 12]     [64, 32, 12, 12

In [54]:
CLASS_NUM = len(classes)

layerlist = [
    #torch.nn.AdaptiveAvgPool2d((1664, 7, 7)),
    #torch.nn.Flatten(),
    torch.nn.Linear(in_features=1664, out_features=256),
    torch.nn.ReLU(),
    torch.nn.Dropout1d(p=0.3),
    torch.nn.Linear(in_features=256, out_features=1024),
    torch.nn.ReLU(),
    torch.nn.Dropout1d(p=0.5),
    torch.nn.Linear(in_features=1024, out_features=512),
    torch.nn.ReLU(),
    torch.nn.Dropout1d(p=0.5),
    torch.nn.Linear(in_features=512, out_features=CLASS_NUM),
    torch.nn.Softmax()
]

model.classifier = torch.nn.Sequential(
    #torch.nn.AdaptiveAvgPool2d(1664),
    #torch.nn.Flatten(),
    torch.nn.Linear(in_features=1664, out_features=256),
    torch.nn.ReLU(),
    torch.nn.Dropout1d(p=0.3),
    torch.nn.Linear(in_features=256, out_features=1024),
    torch.nn.ReLU(),
    torch.nn.Dropout1d(p=0.5),
    torch.nn.Linear(in_features=1024, out_features=512),
    torch.nn.ReLU(),
    torch.nn.Dropout1d(p=0.5),
    torch.nn.Linear(in_features=512, out_features=CLASS_NUM),
    torch.nn.Softmax()
    ).to(device)

# def classifier_forward(self, x):
#     x = self.classifier_dropout1(F.relu(self.classifier_linear1(x)))
#     x = self.classifier_dropout2(F.relu(self.classifier_linear2(x)))
#     x = self.classifier_dropout3(F.relu(self.classifier_linear3(x)))
#     x = F.softmax(self.classifier_linear4(x))
#     return x

# model.classifier.forward = classifier_forward

summary(
    model=model,
    input_size=(BATCH_SIZE, 3, IMG_HEIGHT, IMG_WIDTH),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)


c:\Users\Nevena\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
DenseNet (DenseNet)                           [64, 3, 48, 48]      [64, 7]              --                   Partial
├─Sequential (features)                       [64, 3, 48, 48]      [64, 1664, 1, 1]     --                   False
│    └─Conv2d (conv0)                         [64, 3, 48, 48]      [64, 64, 24, 24]     (9,408)              False
│    └─BatchNorm2d (norm0)                    [64, 64, 24, 24]     [64, 64, 24, 24]     (128)                False
│    └─ReLU (relu0)                           [64, 64, 24, 24]     [64, 64, 24, 24]     --                   --
│    └─MaxPool2d (pool0)                      [64, 64, 24, 24]     [64, 64, 12, 12]     --                   --
│    └─_DenseBlock (denseblock1)              [64, 64, 12, 12]     [64, 256, 12, 12]    --                   False
│    │    └─_DenseLayer (denselayer1)         [64, 64, 12, 12]     [64, 32, 12, 

In [55]:
def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device) -> Tuple[float, float]:

    model.train()
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        y_pred_class = torch.argmax(y_pred, dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc


def val_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device: torch.device) -> Tuple[float, float, torch.Tensor]:

    model.eval()
    val_loss, val_acc = 0, 0
    y_preds = []

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            val_pred_logits = model(X)
            loss = loss_fn(val_pred_logits, y)
            val_loss += loss.item()

            val_pred_labels = torch.argmax(val_pred_logits, dim=1)
            val_acc += ((val_pred_labels == y).sum().item() / len(val_pred_labels))
            y_preds.append(val_pred_labels.cpu())

    val_loss = val_loss / len(dataloader)
    val_acc = val_acc / len(dataloader)

    y_pred_tensor = torch.cat(y_preds)

    return val_loss, val_acc, y_pred_tensor


def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    val_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    epochs: int,
    device: torch.device) -> Tuple[Dict, torch.Tensor]:

    results = {"train_loss": [],
             "train_acc": [],
             "val_loss": [],
             "val_acc": []}

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device
        )

        val_loss, val_acc, y_preds = val_step(
            model=model,
            dataloader=val_dataloader,
            loss_fn=loss_fn,
            device=device
        )

        print(f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Train acc: {train_acc:.3f}, Val loss: {val_loss:.3f}, Val acc: {val_acc:.3f}")

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["val_loss"].append(val_loss)
        results["val_acc"].append(val_acc)

    return results, y_preds

## Run 

In [56]:
torch.cuda.manual_seed(42)
torch.manual_seed(42)

loss_fn = nn.CrossEntropyLoss()


optimizer = torch.optim.SGD(
    params=model.parameters(),
    lr=0.1,
    weight_decay=0.01
)

data_path = Path("Kaggle/CV/Emotions_detection/emotions_dataset")

train_dataloader, val_dataloader = make_dataloaders(data_path, BATCH_SIZE)

start_time = timer()

model_results, preds = train(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=EPOCHS,
    device=device
)

end_time = timer()
print(f"Total learning time: {(end_time - start_time):.3f}")

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0, Train loss: 1.928, Train acc: 0.250, Val loss: 1.920, Val acc: 0.245
Epoch: 1, Train loss: 1.916, Train acc: 0.251, Val loss: 1.911, Val acc: 0.245
Epoch: 2, Train loss: 1.910, Train acc: 0.251, Val loss: 1.909, Val acc: 0.245
Epoch: 3, Train loss: 1.908, Train acc: 0.251, Val loss: 1.909, Val acc: 0.245
Epoch: 4, Train loss: 1.908, Train acc: 0.251, Val loss: 1.909, Val acc: 0.245
Epoch: 5, Train loss: 1.908, Train acc: 0.251, Val loss: 1.909, Val acc: 0.245
Epoch: 6, Train loss: 1.908, Train acc: 0.251, Val loss: 1.909, Val acc: 0.245
Epoch: 7, Train loss: 1.908, Train acc: 0.251, Val loss: 1.908, Val acc: 0.245
Epoch: 8, Train loss: 1.908, Train acc: 0.251, Val loss: 1.908, Val acc: 0.245
Epoch: 9, Train loss: 1.907, Train acc: 0.251, Val loss: 1.909, Val acc: 0.245
Epoch: 10, Train loss: 1.908, Train acc: 0.251, Val loss: 1.909, Val acc: 0.245
Epoch: 11, Train loss: 1.907, Train acc: 0.251, Val loss: 1.909, Val acc: 0.245
Epoch: 12, Train loss: 1.908, Train acc: 0.251, Va

KeyboardInterrupt: 

### 2.3 Fine-tuning

In [ ]:
optimizer = torch.optim.SGD(
    params=model.parameters(),
    lr=0.001,
    weight_decay=0.01
)

for param in model.parameters():
    param.requires_grad = True


start_time = timer()

model_finetuning_results, finetuned_preds = train(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=FINE_TUNING_EPOCHS,
    device=device
)

end_time = timer()
print(f"Total learning time: {(end_time - start_time):.3f}")

## Validacija

In [ ]:
def plot_curves(results: Dict[str, List[float]]) -> None:
    """Plots loss and accuracy from a results dictionary."""

    train_loss = results["train_loss"]
    val_loss = results["val_loss"]

    train_accuracy = results["train_acc"]
    val_accuracy = results["val_acc"]

    epochs = range(len(results["train_loss"]))

    plt.figure(figsize=(15, 7))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, label="train_loss")
    plt.plot(epochs, val_loss, label="val_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracy, label="train_accuracy")
    plt.plot(epochs, val_accuracy, label="val_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()


plot_curves(model_results)

In [ ]:
def makeClassificationReport(pred, modelName, target_test):
    cm = confusion_matrix(target_test, pred, normalize="pred")
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
    plt.grid(visible=False)
    plt.title(f"Confusion Matrix - {modelName}")
    plt.tick_params(axis=u'both', which=u'both',length=0)
    plt.grid(visible=None)
    plt.savefig(f"graphs/confusionMatrix{modelName}.jpg")
    #plt.show()

    report = classification_report(target_test,pred, digits=5)
    with io.open(f'evaluation/classification_report{modelName}.txt','w',encoding='utf-8') as f: f.write(report)